# Balance Calculators

In [1]:
from pybalance.utils.balance_calculators import *
from pybalance.utils import MatchingData
from pybalance.sim import load_paper_dataset

In [2]:
m =load_paper_dataset()
m

In [13]:
m.counts()

N
population        
pool        250000
target       25000

## Fit Balance Calculator

In [14]:
# Balance calculators in general are "fit" to the whole population data
# Fitting here means fitting preprocessors (e.g. what bins to use when binning
# is involved). It's important to fit once so that all calls to distance()
# can be compared meaningfully.
beta = BetaBalance(m)
target, pool = split_target_pool(m)

## Balance between pool and target

In [15]:
beta.distance(pool)

tensor(0.2353, dtype=torch.float64)

In [16]:
# Specifying target is optional
beta.distance(pool, target)

tensor(0.2353, dtype=torch.float64)

## Balance between subset of pool and target

In [17]:
beta.distance(pool.sample(n=100))

tensor(0.2366, dtype=torch.float64)

In [18]:
# Can also take subsets of the target
beta.distance(pool.sample(n=100), target.sample(n=100))

tensor(0.2669, dtype=torch.float64)

## Balance between several subsets simultaneously

In [19]:
pool_subsets = np.array([
    np.random.choice(pool.reset_index().index.values, size=100, replace=False),
    np.random.choice(pool.reset_index().index.values, size=100, replace=False)
])
beta.distance(pool_subsets)

tensor([0.2404, 0.2418], dtype=torch.float64)

In [9]:
pool_subsets = [
    np.random.choice(pool.reset_index().index.values, size=100, replace=False),
    np.random.choice(pool.reset_index().index.values, size=100, replace=False)
]
target_subsets = [
    np.random.choice(target.reset_index().index.values, size=100, replace=False),
    np.random.choice(target.reset_index().index.values, size=100, replace=False)
]
beta.distance(pool_subsets, target_subsets)

/Users/gmema/src/pybalance/pybalance/utils/balance_calculators.py:224: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  subset_populations = torch.tensor(


tensor([0.2602, 0.2757], dtype=torch.float64)

In [28]:
# Must have same number of subsets! This will throw an error:
pool_subsets = [
    np.random.choice(pool.reset_index().index.values, size=100, replace=False),
    np.random.choice(pool.reset_index().index.values, size=100, replace=False)
]
target_subsets = [
    np.random.choice(target.reset_index().index.values, size=100, replace=False),
    np.random.choice(target.reset_index().index.values, size=100, replace=False),
    np.random.choice(target.reset_index().index.values, size=100, replace=False)
]
try:
    beta.distance(pool_subsets, target_subsets)
except ValueError as e:
    print(e)

Number of subset populations must be same for pool and target!


## Basic Genetic Optimizer

Here is a very basic, un-optimized implementation of genetic matching! It's not very smart, because it doesn't mix the good populations. This is just an illustration of using the balance calculator.

In [27]:
def get_subsets(pool, target, pool_size, target_size, n_subsets):
    pool = pool.reset_index()
    target = target.reset_index()
    
    pool_subsets = [
        np.random.choice(pool.index.values, size=pool_size, replace=False) for _ in range(n_subsets)
    ]
    target_subsets = [
        np.random.choice(target.index.values, size=target_size, replace=False) for _ in range(n_subsets)
    ]
    return pool_subsets, target_subsets


pool_size = 1000
target_size = 1000
n_subsets = 100
best_match = None
best_distance = 100000
for j in range(100):
    pool_subsets, target_subsets = get_subsets(pool, target, pool_size, target_size, n_subsets)
    distances = beta.distance(pool_subsets, target_subsets)
    this_best_distance = distances.min()
    if this_best_distance < best_distance:
        best_distance = this_best_distance
        best_match_idx = distances.argmin()
        best_match = pool_subsets[best_match_idx], target_subsets[best_match_idx]

    if not j % 10:
        print(f'Generation {j} / Best distance found {best_distance:.3f}')

Generation 0 / Best distance found 0.215
Generation 10 / Best distance found 0.215
Generation 20 / Best distance found 0.211
Generation 30 / Best distance found 0.211
Generation 40 / Best distance found 0.208
Generation 50 / Best distance found 0.208
Generation 60 / Best distance found 0.208
Generation 70 / Best distance found 0.208
Generation 80 / Best distance found 0.208
Generation 90 / Best distance found 0.208
